In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.datasets import load_digits

In [2]:
dataset = pd.read_csv('/Users/pawankumarkc/Documents/vscode_workspace/MLAlgo/datasets/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
dataset['Exited'].value_counts()

#Imbalanced data

0    7963
1    2037
Name: Exited, dtype: int64

In [4]:
dataset = dataset.iloc[:,3:]
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
dataset.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [8]:
dataset = pd.get_dummies(dataset, columns=['Geography','Gender'], drop_first=True)

In [9]:
dataset.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [13]:
## Split the data into dep and ind variable

x = dataset.drop(['Exited'], axis=1) 
y = dataset[['Exited']]

In [15]:
# Balance the data

from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_smote, y_smote = smote.fit_resample(x, y)
print(y.value_counts())
print(y_smote.value_counts())

Exited
0         7963
1         2037
dtype: int64
Exited
0         7963
1         7963
dtype: int64


In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_smote, y_smote, test_size=0.25, random_state=101)

In [17]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [18]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

In [19]:
x.shape

(10000, 11)

In [21]:
# Multi layer preceptron

model = Sequential()
model.add(Dense(32, activation='relu',kernel_regularizer=keras.regularizers.L2(0.03), input_dim=11, kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu', kernel_regularizer=keras.regularizers.L2(0.03), kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 32)                384       
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 batch_normalization_2 (Bat  (None, 32)                128       
 chNormalization)                                                
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 batch_normalization_3 (Bat  (None, 32)                128       
 chNormalization)                                     

In [22]:
model.get_weights()

[array([[ 0.04503708, -0.6734913 ,  0.678027  , -0.49905542, -0.31851572,
          0.796708  , -0.14913155, -0.58703357,  0.51083094, -0.45684192,
          0.1533386 , -0.4644907 , -0.4435257 , -0.608214  ,  0.00346696,
         -0.4436048 , -0.75879055,  0.10924438, -0.3491522 , -0.9213422 ,
          0.68189335,  0.78235763, -0.0838723 , -0.00746389, -0.30139804,
         -0.40382287,  0.3115964 ,  0.01014484,  0.5991299 , -0.1468734 ,
          0.20460491, -0.76793474],
        [-0.66952085,  0.65255094, -0.6798052 ,  0.21515581, -0.7898342 ,
          0.26504737,  0.06494351, -0.40577775,  0.21561325, -0.5422905 ,
         -0.29254603,  0.6956811 , -0.06266653,  0.32070553, -0.32673153,
         -0.20136395,  0.24442363,  0.7540886 ,  0.09994335,  0.09731679,
         -0.4288791 ,  0.26053378,  0.8079847 ,  0.31594422, -0.2237311 ,
         -0.73279786, -0.53164554, -0.16122301, -0.439185  , -0.3153363 ,
          0.44520608, -0.12606539],
        [ 0.08316143,  0.46590757, -0.91

In [26]:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
history = model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
374/374 [==============================] - 1s 762us/step - loss: 2.8694 - accuracy: 0.5806 - val_loss: 1.5057 - val_accuracy: 0.7504
Epoch 2/100
374/374 [==============================] - 0s 549us/step - loss: 1.0674 - accuracy: 0.7223 - val_loss: 0.7202 - val_accuracy: 0.7906
Epoch 3/100
374/374 [==============================] - 0s 551us/step - loss: 0.6424 - accuracy: 0.7648 - val_loss: 0.5299 - val_accuracy: 0.7938
Epoch 4/100
374/374 [==============================] - 0s 552us/step - loss: 0.5305 - accuracy: 0.7811 - val_loss: 0.4737 - val_accuracy: 0.8059
Epoch 5/100
374/374 [==============================] - 0s 551us/step - loss: 0.5047 - accuracy: 0.7945 - val_loss: 0.4532 - val_accuracy: 0.8139
Epoch 6/100
374/374 [==============================] - 0s 557us/step - loss: 0.4885 - accuracy: 0.7981 - val_loss: 0.4351 - val_accuracy: 0.8252
Epoch 7/100
374/374 [==============================] - 0s 549us/step - loss: 0.4806 - accuracy: 0.8010 - val_loss: 0.4352 - val_ac

In [ ]:
# Remove dropout and batch normalization, etc to check model score improvements
# Multi layer preceptron

model = Sequential()
model.add(Dense(32, activation='relu',kernel_regularizer=keras.regularizers.L2(0.03), input_dim=11, kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu', kernel_regularizer=keras.regularizers.L2(0.03), kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))

model.summary()